In [1]:
import urllib.request,sys,time
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re
import os
import numpy as np
from requests import Session
from multiprocessing.dummy import Pool as ThreadPool
import concurrent.futures

In [9]:
s = Session()
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) '\
                         'AppleWebKit/537.36 (KHTML, like Gecko) '\
                         'Chrome/75.0.3770.80 Safari/537.36'}
# Add headers
s.headers.update(headers)


In [10]:
def get_article_items(category,pages):
  pagesToGet = pages
  data = pd.DataFrame([])
  for page in range(1,pagesToGet+1):
      url = category + '/page/' + str(page)
      #an exception might be thrown, so the code should be in a try-except block
      try:
          #use the browser to get the url. This is suspicious command that might blow up.
          page=s.get(url)                             # this might throw an exception if something goes wrong.
      except Exception as e:                                   # this describes what to do if an exception is thrown
          error_type, error_obj, error_info = sys.exc_info()      # get the exception information
          print ('ERROR FOR LINK:',url)                          #print the link that cause the problem
          print (error_type, 'Line:', error_info.tb_lineno)     #print error info and line that threw the exception
          continue                                              #ignore this page. Abandon this and go back. 
      soup=BeautifulSoup(page.text,'lxml')
      links = soup.find_all('li',attrs={"class":"lx-stream__post-container"})
      for i in links:
          try: 
              title = i.find("span",attrs={'class':'lx-stream-post__header-text gs-u-align-middle'}).text.strip()
              link = "https://www.bbc.com"
              link += i.find('a',attrs={"class":"qa-heading-link lx-stream-post__header-link"})["href"]
              date = i.find('div',attrs={"class":"gs-u-display-flex"}).find("span",attrs={"class":"qa-post-auto-meta"}).text
              data = data.append(pd.DataFrame({'title': title, 'url': link,'date' : date}, index=[0]), ignore_index=True)
          except:
              continue
  return data
    

    

In [6]:
%%time
page1 = get_article_items("https://www.bbc.com/vietnamese/topics/ckdxnx1x5rnt",1) #url = category url, pages = amount of pages available on category page
page1["category"] = "test"

ERROR FOR LINK: https://www.bbc.com/vietnamese/topics/ckdxnx1x5rnt/page/1
<class 'NameError'> Line: 9
CPU times: user 1.58 ms, sys: 95 µs, total: 1.67 ms
Wall time: 1.59 ms


In [8]:
def get_user_need(page):
    try:
        article = s.get(page)
        soup = BeautifulSoup(article.content, 'lxml')
        x = [i for i in soup.find("script",text = re.compile(r'(?<="campaignName".").+?(?=\")'))][0]
        return re.search(r'(?<="campaignName".").+?(?=\")',x)[0]
    except:
        return np.nan
    
def get_text(url):
    try:
        article = s.get(url)
        soup = BeautifulSoup(article.content, 'lxml')
        body = soup.find(role='main')
        text = [p.text for p in body.find_all('p',dir="ltr")]
        return ''.join(text)
    except:
        return np.nan


In [ ]:
%%time
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = []
    futures2 = []
    for page in df["url"]:
        futures.append(executor.submit(get_user_need, page))
        futures2.append(executor.submit(get_text, page))

In [ ]:
userneed = [df.result() for df in futures]
df["User Need"] = userneed
df["User Need"] = df["User Need"].apply(lambda s: re.sub(r"(WS - )", "", s) if pd.notnull(s) else s)

In [ ]:
text = [df.result() for df in futures2]
df["text"] = text